## Setup

### Imports and installations

In [ ]:
%load_ext rpy2.ipython


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade spacy

     |████████████████████████████████| 5.9 MB 5.5 MB/s 
     |████████████████████████████████| 627 kB 37.7 MB/s 
     |████████████████████████████████| 451 kB 46.4 MB/s 
     |████████████████████████████████| 10.1 MB 31.4 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


[link text](https://)

In [ ]:
!python -m spacy download en_core_web_lg
# typical guidance is to restart runtime after downloading

     |████████████████████████████████| 777.1 MB 16 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import pandas as pd
import os
from random import sample

In [ ]:
proj_dir = "/content/drive/Shareddrives/Working Group - NLP in Engineering Education Research/Fall 2021 Independent Study/De-identifier project"
os.chdir(proj_dir)
os.listdir()


['De-identifier test.ipynb',
 'baby-names_hadley_20211022.txt',
 'world-cities_csv.csv',
 'baby-names_hadley_20211022.csv',
 'world-universities.txt',
 'world-universities.csv',
 'name_results_df_20211022.csv',
 'name_post1950_results_df_20211022.csv',
 'name_post1950_full_rn_results_df_20211023.csv',
 'name_post1960_full_rn_results_df_20211023.csv',
 'De-identifier project.Rproj',
 '.Rproj.user',
 'baby_names_ranking_df.csv',
 'name_post1950_500_rn_results_df_20211023.csv',
 'most-common-name_surnames.csv',
 'name_results_df_20211023.csv',
 'ASEE 2022',
 'full_name_post1960_5000_rn_results_df_20211024.csv',
 'deidentifier tests.Rmd',
 '.Rhistory',
 'Remove name dataset.gsheet']

In [ ]:
import spacy

In [ ]:
def remove_name(text, check_name=True, check_org=True, check_gpe=True):
    # nlp = spacy.load("en_core_web_lg")

    doc = nlp(text)
    
    name_list = [ent.text for ent in doc.ents if ent.label_ == "PERSON"] 

    org_list = [ent.text for ent in doc.ents if ent.label_ == "ORG"]

    gpe_list = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
    #print(names)  

    if name_list or org_list or gpe_list:
        new_txt = text
        if check_name:
            for name in name_list:
                new_txt = new_txt.replace(name, "***")
                #print(new_txt)
        
        if check_org:
            for org in org_list:
                new_txt = new_txt.replace(org, "[[[")

        if check_gpe:
            for gpe in gpe_list:
                new_txt = new_txt.replace(gpe, "---")

        return new_txt
    
    else:
        return text


In [ ]:
"""
test_2 = "The Easter egg and ham sandwich from Jordan was pretty okay."

doc = nlp(test_2)
"""

In [ ]:
"""
for token in doc:
  print(token.text)
  print(token.pos_)

for ent in doc.ents:
  print(ent.text)
  print(ent.label_)
  """

### Read in the sentences google sheet

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('Remove name dataset').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
sentence_df = pd.DataFrame.from_records(rows)

[['Author', 'Sentence', 'sentence type', 'Entered Sentence', 'Final Sentence', '', 'legend', '***', 'names'], ['Andrew', "I didn't like being in the same group as ***.", 'name_only', '', '', '', '', '---', 'geoplitical entities'], ['Andrew', 'Dr. *** has a weird way of talking to the rest of the class', 'name_only', '', '', '', '', '[[[', 'organizations'], ['Andrew', 'I thought *** was a great teacher', 'name_only', '', '', '', '', '', ''], ['Andrew', '*** works hard in class and always shows up on time', 'name_only', '', '', '', '', '', ''], ['Andrew', "I really wish that *** wasn't on my team", 'name_only', '', '', '', '', '', ''], ['Andrew', "The note's from ***'s class were great", 'name_only', '', '', '', '', '', ''], ['Andrew', "I was told by *** that we didn't need to finish the lab reports", 'name_only', '', '', '', '', '', ''], ['Andrew', 'I went to office hours ever week and got help from ***', 'name_only', '', '', '', '', '', ''], ['Andrew', 'The textbook by *** was tough to

In [ ]:
sentence_df

,0,1,2,3,4,5,6,7,8
0,Author,Sentence,sentence type,Entered Sentence,Final Sentence,,legend,***,names
1,Andrew,I didn't like being in the same group as ***.,name_only,,,,,---,geoplitical entities
2,Andrew,Dr. *** has a weird way of talking to the rest...,name_only,,,,,[[[,organizations
3,Andrew,I thought *** was a great teacher,name_only,,,,,,
4,Andrew,*** works hard in class and always shows up on...,name_only,,,,,,
...,...,...,...,...,...,...,...,...,...
396,Yasir,,name_gpe_org,,,,,,
397,Yasir,,name_gpe_org,,,,,,
398,Yasir,,name_gpe_org,,,,,,
399,Yasir,,name_gpe_org,,,,,,


In [ ]:
## need to reset the top row

new_header = sentence_df.iloc[0] #grab the first row for the header
sentence_df = sentence_df[1:] #take the data less the header row
sentence_df.columns = new_header #set the header row as the df header

In [ ]:
sentence_df

,Author,Sentence,sentence type,Entered Sentence,Final Sentence,,legend,***,names
1,Andrew,I didn't like being in the same group as ***.,name_only,,,,,---,geoplitical entities
2,Andrew,Dr. *** has a weird way of talking to the rest...,name_only,,,,,[[[,organizations
3,Andrew,I thought *** was a great teacher,name_only,,,,,,
4,Andrew,*** works hard in class and always shows up on...,name_only,,,,,,
5,Andrew,I really wish that *** wasn't on my team,name_only,,,,,,
...,...,...,...,...,...,...,...,...,...
396,Yasir,,name_gpe_org,,,,,,
397,Yasir,,name_gpe_org,,,,,,
398,Yasir,,name_gpe_org,,,,,,
399,Yasir,,name_gpe_org,,,,,,


Pull just the sentences for names only

In [ ]:
sentence_df = sentence_df[["Sentence", "sentence type"]]

In [ ]:
test_df = sentence_df[sentence_df["sentence type"] == "name_only"]

In [ ]:
# remove the empty cells missing sentences
test_df = test_df[test_df['Sentence'].str.strip().astype(bool)]

In [ ]:
test_df

,Sentence,sentence type
1,I didn't like being in the same group as ***.,name_only
2,Dr. *** has a weird way of talking to the rest...,name_only
3,I thought *** was a great teacher,name_only
4,*** works hard in class and always shows up on...,name_only
5,I really wish that *** wasn't on my team,name_only
...,...,...
316,I have my advisor meeting with *** and *** tom...,name_only
317,There have been many intelligent people in eng...,name_only
318,They are going for a walk with *** for lunch a...,name_only
319,I am running out of comments that I can make a...,name_only


### Import first names csv

This csv comes from the Social Security Administration via https://github.com/hadley/data-baby-names. It is a list of the top 1000 baby names each year from 1880 to 2008

In [ ]:
names_df = pd.read_csv("baby-names_hadley_20211022.csv")

In [ ]:
print(names_df.columns)
print(names_df.shape)

Index(['year', 'name', 'percent', 'sex'], dtype='object')
(258000, 4)


### Import surnames csv

In [ ]:
surnames_df = pd.read_csv("most-common-name_surnames.csv")

In [ ]:
print(surnames_df.columns)
print(surnames_df.shape)

Index(['rowid', 'name', 'rank', 'count', 'prop100k', 'cum_prop100k',
       'pctwhite', 'pctblack', 'pctapi', 'pctaian', 'pct2prace',
       'pcthispanic'],
      dtype='object')
(1000, 12)


In [ ]:
surnames_df

,rowid,name,rank,count,prop100k,cum_prop100k,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,1,SMITH,1,2376206,880.85,880.85,73.35,22.22,0.40,0.85,1.63,1.56
1,2,JOHNSON,2,1857160,688.44,1569.30,61.55,33.8,0.42,0.91,1.82,1.50
2,3,WILLIAMS,3,1534042,568.66,2137.96,48.52,46.72,0.37,0.78,2.01,1.60
3,4,BROWN,4,1380145,511.62,2649.58,60.71,34.54,0.41,0.83,1.86,1.64
4,5,JONES,5,1362755,505.17,3154.75,57.69,37.73,0.35,0.94,1.85,1.44
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,LUTZ,996,31940,11.84,40522.49,96.18,0.54,0.44,0.23,0.95,1.64
996,997,DUARTE,997,31896,11.82,40534.31,18.08,1.31,1.02,0.3,2.73,76.56
997,998,KIDD,998,31886,11.82,40546.13,76.98,18.85,0.84,0.52,1.65,1.16
998,999,KEY,999,31882,11.82,40557.95,71.31,23.7,0.72,0.9,1.68,1.69


In [ ]:
# make sure able to capitalize only first letter of surname since dataset has all letters capitalized
test_str = "BROWN"
test_str.lower().capitalize()

'Brown'

## Test - Top 1000 names since 1880

In [ ]:
names_list = names_df.name.to_list()

Use all names in the list (dating back to 1880)

In [ ]:
len(names_list)

258000

Select only the unique names from the names list

In [ ]:
names_set = set(names_list)

In [ ]:
names_list = list(names_set)

In [ ]:
len(names_list) # only 6782 unique names

6782

In [ ]:
# short sentence list to make sure test cycle working

#test_sent_df = sentence_df.head(10)
#test_sent_df

,Sentence,sentence type
1,I didn't like being in the same group as ***.,name_only
2,Dr. *** has a weird way of talking to the rest...,name_only
3,I thought *** was a great teacher,name_only
4,*** works hard in class and always shows up on...,name_only
5,I really wish that *** wasn't on my team,name_only
6,The note's from ***'s class were great,name_only
7,I was told by *** that we didn't need to finis...,name_only
8,I went to office hours ever week and got help ...,name_only
9,The textbook by *** was tough to read,name_only
10,He told *** that she was going to need a new p...,name_only


In [ ]:
# short test to make sure test cycle works

#test_names_list = names_list[-10:]
#test_names_list

['Helene',
 'Quinton',
 'Kendall',
 'Johny',
 'Carmelo',
 'Thedore',
 'Adda',
 'Carl',
 'Adah',
 'Manuel']

### Test section

We will use the following code to test the name removal process

#### Creating the Names Only dataset



In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
names_dict = {'name': [],
              'original_sentence': [],
              'named_sentence':[],
              'final_sentence':[]}

for index, row in test_df.iterrows():
  for name in names_list:


    original_sentence = row['Sentence']
    named_sentence = original_sentence.replace("***", name)

    final_sentence = remove_name(named_sentence, check_name=True, check_gpe=False, check_org=False)

    names_dict['name'].append(name)
    names_dict['original_sentence'].append(original_sentence)
    names_dict['named_sentence'].append(named_sentence)
    names_dict['final_sentence'].append(final_sentence)


result_df = pd.DataFrame(names_dict)

In [ ]:
result_df.final_sentence.to_list()

["I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Nelda.",
 "I didn't like being in the same group as Josefa.",
 "I didn't like being in the same group as Louella.",
 "I didn't like being in the same group as Minta.",
 "I didn't like being in the same group as Catina.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Crysta.",
 "I didn't like being in the same group as Mahlon.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Marnita.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Lute.",
 "I didn't like being in the same group as Sunshine.",
 "I didn't like bein

In [ ]:
result_df.to_csv("name_results_df_20211023.csv")

### Results analysis - all names since 1880

In [ ]:
%%R
library(tidyverse)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
%%R 
results_df <- read_csv("name_results_df_20211023.csv")

R[write to console]: New names:
* `` -> ...1



Rows: 542560 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): name, original_sentence, named_sentence, final_sentence
dbl (1): ...1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
%%R 
results_df

# A tibble: 542,560 × 5
    ...1 name     original_sentence     named_sentence       final_sentence     
   <dbl> <chr>    <chr>                 <chr>                <chr>              
 1     0 Jaylynn  I didn't like being … I didn't like being… I didn't like bein…
 2     1 Nelda    I didn't like being … I didn't like being… I didn't like bein…
 3     2 Josefa   I didn't like being … I didn't like being… I didn't like bein…
 4     3 Louella  I didn't like being … I didn't like being… I didn't like bein…
 5     4 Minta    I didn't like being … I didn't like being… I didn't like bein…
 6     5 Catina   I didn't like being … I didn't like being… I didn't like bein…
 7     6 Marquita I didn't like being … I didn't like being… I didn't like bein…
 8     7 Crysta   I didn't like being … I didn't like being… I didn't like bein…
 9     8 Mahlon   I didn't like being … I didn't like being… I didn't like bein…
10     9 Carleen  I didn't like being … I didn't like being… I didn't like bein…
# … 

In [ ]:
%%R

results_df <- results_df %>%
mutate(name_removed_flag = case_when(str_detect(final_sentence, "\\*\\*\\*") == 1 ~ 1, 
                                     str_detect(final_sentence, "\\*\\*\\*") == 0 ~ 0))

In [ ]:
%%R 
results_df

# A tibble: 542,560 × 6
    ...1 name     original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>    <chr>            <chr>          <chr>                     <dbl>
 1     0 Jaylynn  I didn't like b… I didn't like… I didn't like…                1
 2     1 Nelda    I didn't like b… I didn't like… I didn't like…                0
 3     2 Josefa   I didn't like b… I didn't like… I didn't like…                0
 4     3 Louella  I didn't like b… I didn't like… I didn't like…                0
 5     4 Minta    I didn't like b… I didn't like… I didn't like…                0
 6     5 Catina   I didn't like b… I didn't like… I didn't like…                0
 7     6 Marquita I didn't like b… I didn't like… I didn't like…                1
 8     7 Crysta   I didn't like b… I didn't like… I didn't like…                0
 9     8 Mahlon   I didn't like b… I didn't like… I didn't like…                0
10     9 Carleen  I didn't like b… I didn't like… I didn't like…          

In [ ]:
%%R

mean(results_df$name_removed_flag)

[1] 0.7003078


Average score of 70% for all 1000 names per gender per year from 1980-2008

In [ ]:
%%R

failed_names_df <- results_df %>%
filter(name_removed_flag == 0)

In [ ]:
%%R 
failed_names_df

# A tibble: 73,718 × 6
    ...1 name       original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>      <chr>            <chr>          <chr>                     <dbl>
 1     3 Lara       I didn't like b… I didn't like… I didn't like…                0
 2     5 Julius     I didn't like b… I didn't like… I didn't like…                0
 3     8 Elijah     I didn't like b… I didn't like… I didn't like…                0
 4    15 Josiephine I didn't like b… I didn't like… I didn't like…                0
 5    21 Devaughn   I didn't like b… I didn't like… I didn't like…                0
 6    35 Shaquita   I didn't like b… I didn't like… I didn't like…                0
 7    60 Loyce      I didn't like b… I didn't like… I didn't like…                0
 8    79 Omer       I didn't like b… I didn't like… I didn't like…                0
 9    87 Elois      I didn't like b… I didn't like… I didn't like…                0
10   103 Geo        I didn't like b… I didn't like… I

In [ ]:
%%R 
failed_names_df %>%
distinct(name)

# A tibble: 4,333 × 1
   name      
   <chr>     
 1 Lara      
 2 Julius    
 3 Elijah    
 4 Josiephine
 5 Devaughn  
 6 Shaquita  
 7 Loyce     
 8 Omer      
 9 Elois     
10 Geo       
# … with 4,323 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name)


# A tibble: 4,333 × 2
   name         n
   <chr>    <int>
 1 Aaden        7
 2 Aaliyah      1
 3 Aarav       67
 4 Ab          74
 5 Abbey        1
 6 Abbigail     9
 7 Abdiel      17
 8 Abdullah     1
 9 Abel        49
10 Abie        10
# … with 4,323 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name) %>%
arrange(desc(n))

# A tibble: 4,333 × 2
   name        n
   <chr>   <int>
 1 Add        78
 2 Ah         78
 3 April      78
 4 August     78
 5 Author     78
 6 Christ     78
 7 Doctor     78
 8 Else       78
 9 Elwanda    78
10 French     78
# … with 4,323 more rows


## Test top 1000 names since 1950

In [ ]:
names_df.columns

Index(['year', 'name', 'percent', 'sex'], dtype='object')

In [ ]:
names_df

,year,name,percent,sex
0,1880,John,0.081541,boy
1,1880,William,0.080511,boy
2,1880,James,0.050057,boy
3,1880,Charles,0.045167,boy
4,1880,George,0.043292,boy
...,...,...,...,...
257995,2008,Carleigh,0.000128,girl
257996,2008,Iyana,0.000128,girl
257997,2008,Kenley,0.000127,girl
257998,2008,Sloane,0.000127,girl


In [ ]:
names_df[names_df['year'] > 1949].head(30)

,year,name,percent,sex
70000,1950,James,0.047336,boy
70001,1950,Robert,0.045930,boy
70002,1950,John,0.043655,boy
70003,1950,Michael,0.035810,boy
70004,1950,David,0.033382,boy
70005,1950,William,0.033372,boy
70006,1950,Richard,0.028027,boy
70007,1950,Thomas,0.025070,boy
70008,1950,Charles,0.021495,boy
70009,1950,Gary,0.018547,boy


In [ ]:
names_post1950_df = names_df[names_df['year'] > 1949]

In [ ]:
names_list = names_post1950_df.name.to_list()

In [ ]:
len(names_list)

118000

Select only the unique names from the names list

In [ ]:
names_set = set(names_list)

In [ ]:
names_list = list(names_set)

In [ ]:
len(names_list) # only 4682 unique names

4682

In [ ]:
# short sentence list to make sure test cycle working

test_sent_df = sentence_df.head(10)
test_sent_df

,Sentence,sentence type
1,I didn't like being in the same group as ***.,name_only
2,Dr. *** has a weird way of talking to the rest...,name_only
3,I thought *** was a great teacher,name_only
4,*** works hard in class and always shows up on...,name_only
5,I really wish that *** wasn't on my team,name_only
6,The note's from ***'s class were great,name_only
7,I was told by *** that we didn't need to finis...,name_only
8,I went to office hours ever week and got help ...,name_only
9,The textbook by *** was tough to read,name_only
10,He told *** that she was going to need a new p...,name_only


In [ ]:
# short test to make sure test cycle works

#test_names_list = names_list[-10:]
#test_names_list

#### Test section

We will use the following code to test the name removal process

#### Creating the Names Only dataset for post 1950 names and nlp = spacy.load("en_core_web_lg")



In [ ]:
# Test the remove names function with either nlp = spacy.load("en_core_web_lg") or nlp = English()

In [ ]:
nlp = spacy.load("en_core_web_lg")
#nlp = English()

In [ ]:
names_dict = {'name': [],
              'original_sentence': [],
              'named_sentence':[],
              'final_sentence':[]}

for index, row in test_df.iterrows():
  for name in names_list:


    original_sentence = row['Sentence']
    named_sentence = original_sentence.replace("***", name)

    final_sentence = remove_name(named_sentence, check_name = True, check_org = False, check_gpe = False)

    names_dict['name'].append(name)
    names_dict['original_sentence'].append(original_sentence)
    names_dict['named_sentence'].append(named_sentence)
    names_dict['final_sentence'].append(final_sentence)


result_df = pd.DataFrame(names_dict)

In [ ]:
result_df.final_sentence.to_list()

["I didn't like being in the same group as Sanjuanita.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Ariel.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Lorelai.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Elijah.",
 "I didn't like being in the same group as Zakary.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the sa

In [ ]:
result_df.to_csv("name_post1950_full_rn_results_df_20211023.csv")

### Results analysis

In [ ]:
%%R
library(tidyverse)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
%%R 
results_df <- read_csv("name_post1950_full_rn_results_df_20211023.csv")

R[write to console]: New names:
* `` -> ...1



Rows: 374560 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): name, original_sentence, named_sentence, final_sentence
dbl (1): ...1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
%%R 
results_df

# A tibble: 374,560 × 5
    ...1 name       original_sentence    named_sentence      final_sentence     
   <dbl> <chr>      <chr>                <chr>               <chr>              
 1     0 Sanjuanita I didn't like being… I didn't like bein… I didn't like bein…
 2     1 Brock      I didn't like being… I didn't like bein… I didn't like bein…
 3     2 Deborrah   I didn't like being… I didn't like bein… I didn't like bein…
 4     3 Paulo      I didn't like being… I didn't like bein… I didn't like bein…
 5     4 Brennen    I didn't like being… I didn't like bein… I didn't like bein…
 6     5 Damian     I didn't like being… I didn't like bein… I didn't like bein…
 7     6 Ariel      I didn't like being… I didn't like bein… I didn't like bein…
 8     7 Rogers     I didn't like being… I didn't like bein… I didn't like bein…
 9     8 Astrid     I didn't like being… I didn't like bein… I didn't like bein…
10     9 Lorelai    I didn't like being… I didn't like bein… I didn't like bein…
# … 

In [ ]:
%%R

results_df <- results_df %>%
mutate(name_removed_flag = case_when(str_detect(final_sentence, "\\*\\*\\*|\\[\\[\\[|\\-\\-\\-") == 1 ~ 1, 
                                     str_detect(final_sentence, "\\*\\*\\*|\\[\\[\\[") == 0 ~ 0))

In [ ]:
%%R 
results_df

# A tibble: 374,560 × 6
    ...1 name       original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>      <chr>            <chr>          <chr>                     <dbl>
 1     0 Sanjuanita I didn't like b… I didn't like… I didn't like…                0
 2     1 Brock      I didn't like b… I didn't like… I didn't like…                1
 3     2 Deborrah   I didn't like b… I didn't like… I didn't like…                1
 4     3 Paulo      I didn't like b… I didn't like… I didn't like…                1
 5     4 Brennen    I didn't like b… I didn't like… I didn't like…                1
 6     5 Damian     I didn't like b… I didn't like… I didn't like…                1
 7     6 Ariel      I didn't like b… I didn't like… I didn't like…                0
 8     7 Rogers     I didn't like b… I didn't like… I didn't like…                1
 9     8 Astrid     I didn't like b… I didn't like… I didn't like…                1
10     9 Lorelai    I didn't like b… I didn't like… 

In [ ]:
%%R 
results_df %>% 
filter(name_removed_flag == 0) %>% 
select(final_sentence)

# A tibble: 88,859 × 1
   final_sentence                                      
   <chr>                                               
 1 I didn't like being in the same group as Sanjuanita.
 2 I didn't like being in the same group as Ariel.     
 3 I didn't like being in the same group as Lorelai.   
 4 I didn't like being in the same group as Elijah.    
 5 I didn't like being in the same group as Zakary.    
 6 I didn't like being in the same group as Stanley.   
 7 I didn't like being in the same group as Lara.      
 8 I didn't like being in the same group as India.     
 9 I didn't like being in the same group as Douglass.  
10 I didn't like being in the same group as Franklin.  
# … with 88,849 more rows


In [ ]:
%%R

mean(results_df$name_removed_flag)

[1] 0.7627643


In [ ]:
%%R

failed_names_df <- results_df %>%
filter(name_removed_flag == 0)

In [ ]:
%%R 
failed_names_df

# A tibble: 41,773 × 6
    ...1 name      original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>     <chr>            <chr>          <chr>                     <dbl>
 1     5 Isaiah    I didn't like b… I didn't like… I didn't like…                0
 2     8 Rianna    I didn't like b… I didn't like… I didn't like…                0
 3    10 Katrina   I didn't like b… I didn't like… I didn't like…                0
 4    26 Karlie    I didn't like b… I didn't like… I didn't like…                0
 5    39 Jackeline I didn't like b… I didn't like… I didn't like…                0
 6    69 Cristian  I didn't like b… I didn't like… I didn't like…                0
 7    89 Uriel     I didn't like b… I didn't like… I didn't like…                0
 8    92 Elouise   I didn't like b… I didn't like… I didn't like…                0
 9   124 Karma     I didn't like b… I didn't like… I didn't like…                0
10   158 Jesus     I didn't like b… I didn't like… I didn't like

In [ ]:
%%R 
failed_names_df %>%
distinct(name)

# A tibble: 2,808 × 1
   name     
   <chr>    
 1 Isaiah   
 2 Rianna   
 3 Katrina  
 4 Karlie   
 5 Jackeline
 6 Cristian 
 7 Uriel    
 8 Elouise  
 9 Karma    
10 Jesus    
# … with 2,798 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name)


# A tibble: 2,808 × 2
   name         n
   <chr>    <int>
 1 Aaden        7
 2 Aaliyah      1
 3 Aarav       67
 4 Abbey        1
 5 Abbigail     9
 6 Abdiel      17
 7 Abdullah     1
 8 Abel        49
 9 Abigale     24
10 Abigayle    12
# … with 2,798 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name) %>%
arrange(desc(n))

# A tibble: 2,808 × 2
   name         n
   <chr>    <int>
 1 April       78
 2 August      78
 3 German      78
 4 Heaven      78
 5 January     78
 6 Jesus       78
 7 June        78
 8 Karma       78
 9 Precious    78
10 Summer      78
# … with 2,798 more rows


#### Testing performance with string as lowercase

## Testing when strings are lower case

In [ ]:
nlp = spacy.load("en_core_web_lg")


OSError: ignored

In [ ]:
names_dict = {'name': [],
              'original_sentence': [],
              'named_sentence':[],
              'final_sentence':[]}

for index, row in test_df.iterrows():
  for name in names_list:


    original_sentence = row['Sentence']
    named_sentence = original_sentence.replace("***", name.lower())

    final_sentence = remove_name(named_sentence)

    names_dict['name'].append(name)
    names_dict['original_sentence'].append(original_sentence)
    names_dict['named_sentence'].append(named_sentence)
    names_dict['final_sentence'].append(final_sentence)


result_df = pd.DataFrame(names_dict)

In [ ]:
result_df.final_sentence.to_list()

["I didn't like being in the same group as Toriano.",
 "I didn't like being in the same group as Sharif.",
 "I didn't like being in the same group as Valorie.",
 "I didn't like being in the same group as Candi.",
 "I didn't like being in the same group as Clark.",
 "I didn't like being in the same group as Cale.",
 "I didn't like being in the same group as Reilly.",
 "I didn't like being in the same group as Finnegan.",
 "I didn't like being in the same group as Nena.",
 "I didn't like being in the same group as Wilford.",
 "I didn't like being in the same group as Parrish.",
 "I didn't like being in the same group as Dustin.",
 "I didn't like being in the same group as Walker.",
 "I didn't like being in the same group as Shante.",
 "I didn't like being in the same group as Lauren.",
 "I didn't like being in the same group as Jody.",
 "I didn't like being in the same group as Jerrad.",
 "I didn't like being in the same group as Reta.",
 "I didn't like being in the same group as Rod.",


In [ ]:
result_df.to_csv("name_post1950_English_results_df_20211023.csv")

### Results analysis

In [ ]:
%%R
library(tidyverse)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
%%R 
results_df <- read_csv("name_post1950_results_df_20211022.csv")

R[write to console]: New names:
* `` -> ...1



Rows: 374560 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): name, original_sentence, named_sentence, final_sentence
dbl (1): ...1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
%%R 
results_df

# A tibble: 374,560 × 5
    ...1 name     original_sentence     named_sentence       final_sentence     
   <dbl> <chr>    <chr>                 <chr>                <chr>              
 1     0 Shad     I didn't like being … I didn't like being… I didn't like bein…
 2     1 Leilani  I didn't like being … I didn't like being… I didn't like bein…
 3     2 Jaydon   I didn't like being … I didn't like being… I didn't like bein…
 4     3 Valerie  I didn't like being … I didn't like being… I didn't like bein…
 5     4 Sharon   I didn't like being … I didn't like being… I didn't like bein…
 6     5 Isaiah   I didn't like being … I didn't like being… I didn't like bein…
 7     6 Brody    I didn't like being … I didn't like being… I didn't like bein…
 8     7 Lizbeth  I didn't like being … I didn't like being… I didn't like bein…
 9     8 Rianna   I didn't like being … I didn't like being… I didn't like bein…
10     9 Phylicia I didn't like being … I didn't like being… I didn't like bein…
# … 

In [ ]:
%%R

results_df <- results_df %>%
mutate(name_removed_flag = case_when(str_detect(final_sentence, "\\*\\*\\*") == 1 ~ 1, 
                                     str_detect(final_sentence, "\\*\\*\\*") == 0 ~ 0))

In [ ]:
%%R 
results_df

# A tibble: 374,560 × 6
    ...1 name     original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>    <chr>            <chr>          <chr>                     <dbl>
 1     0 Shad     I didn't like b… I didn't like… I didn't like…                1
 2     1 Leilani  I didn't like b… I didn't like… I didn't like…                1
 3     2 Jaydon   I didn't like b… I didn't like… I didn't like…                1
 4     3 Valerie  I didn't like b… I didn't like… I didn't like…                1
 5     4 Sharon   I didn't like b… I didn't like… I didn't like…                1
 6     5 Isaiah   I didn't like b… I didn't like… I didn't like…                0
 7     6 Brody    I didn't like b… I didn't like… I didn't like…                1
 8     7 Lizbeth  I didn't like b… I didn't like… I didn't like…                1
 9     8 Rianna   I didn't like b… I didn't like… I didn't like…                0
10     9 Phylicia I didn't like b… I didn't like… I didn't like…          

In [ ]:
%%R

mean(results_df$name_removed_flag)

[1] 0.8884745


In [ ]:
%%R

failed_names_df <- results_df %>%
filter(name_removed_flag == 0)

In [ ]:
%%R 
failed_names_df

# A tibble: 41,773 × 6
    ...1 name      original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>     <chr>            <chr>          <chr>                     <dbl>
 1     5 Isaiah    I didn't like b… I didn't like… I didn't like…                0
 2     8 Rianna    I didn't like b… I didn't like… I didn't like…                0
 3    10 Katrina   I didn't like b… I didn't like… I didn't like…                0
 4    26 Karlie    I didn't like b… I didn't like… I didn't like…                0
 5    39 Jackeline I didn't like b… I didn't like… I didn't like…                0
 6    69 Cristian  I didn't like b… I didn't like… I didn't like…                0
 7    89 Uriel     I didn't like b… I didn't like… I didn't like…                0
 8    92 Elouise   I didn't like b… I didn't like… I didn't like…                0
 9   124 Karma     I didn't like b… I didn't like… I didn't like…                0
10   158 Jesus     I didn't like b… I didn't like… I didn't like

In [ ]:
%%R 
failed_names_df %>%
distinct(name)

# A tibble: 2,808 × 1
   name     
   <chr>    
 1 Isaiah   
 2 Rianna   
 3 Katrina  
 4 Karlie   
 5 Jackeline
 6 Cristian 
 7 Uriel    
 8 Elouise  
 9 Karma    
10 Jesus    
# … with 2,798 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name)


# A tibble: 2,808 × 2
   name         n
   <chr>    <int>
 1 Aaden        7
 2 Aaliyah      1
 3 Aarav       67
 4 Abbey        1
 5 Abbigail     9
 6 Abdiel      17
 7 Abdullah     1
 8 Abel        49
 9 Abigale     24
10 Abigayle    12
# … with 2,798 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name) %>%
arrange(desc(n))

# A tibble: 2,808 × 2
   name         n
   <chr>    <int>
 1 April       78
 2 August      78
 3 German      78
 4 Heaven      78
 5 January     78
 6 Jesus       78
 7 June        78
 8 Karma       78
 9 Precious    78
10 Summer      78
# … with 2,798 more rows


## Test top 1000 names since 1960

In [ ]:
names_df.columns

Index(['year', 'name', 'percent', 'sex'], dtype='object')

In [ ]:
names_df

,year,name,percent,sex
0,1880,John,0.081541,boy
1,1880,William,0.080511,boy
2,1880,James,0.050057,boy
3,1880,Charles,0.045167,boy
4,1880,George,0.043292,boy
...,...,...,...,...
257995,2008,Carleigh,0.000128,girl
257996,2008,Iyana,0.000128,girl
257997,2008,Kenley,0.000127,girl
257998,2008,Sloane,0.000127,girl


In [ ]:
names_df[names_df['year'] > 1959].head(30)

,year,name,percent,sex
80000,1960,David,0.039669,boy
80001,1960,Michael,0.038868,boy
80002,1960,James,0.035483,boy
80003,1960,John,0.035145,boy
80004,1960,Robert,0.033415,boy
80005,1960,Mark,0.027121,boy
80006,1960,William,0.022783,boy
80007,1960,Richard,0.020106,boy
80008,1960,Thomas,0.018145,boy
80009,1960,Steven,0.015651,boy


In [ ]:
names_post1960_df = names_df[names_df['year'] > 1959]

In [ ]:
names_list = names_post1960_df.name.to_list()

In [ ]:
len(names_list)

98000

Select only the unique names from the names list

In [ ]:
names_set = set(names_list)

In [ ]:
names_list = list(names_set)

In [ ]:
len(names_list) # only 4682 unique names

4362

In [ ]:
# short sentence list to make sure test cycle working

test_sent_df = sentence_df.head(10)
test_sent_df

,Sentence,sentence type
1,I didn't like being in the same group as ***.,name_only
2,Dr. *** has a weird way of talking to the rest...,name_only
3,I thought *** was a great teacher,name_only
4,*** works hard in class and always shows up on...,name_only
5,I really wish that *** wasn't on my team,name_only
6,The note's from ***'s class were great,name_only
7,I was told by *** that we didn't need to finis...,name_only
8,I went to office hours ever week and got help ...,name_only
9,The textbook by *** was tough to read,name_only
10,He told *** that she was going to need a new p...,name_only


In [ ]:
# short test to make sure test cycle works

#test_names_list = names_list[-10:]
#test_names_list

#### Test section

We will use the following code to test the name removal process

#### Creating the Names Only dataset for post 1950 names and nlp = spacy.load("en_core_web_lg")



In [ ]:
# Test the remove names function with either nlp = spacy.load("en_core_web_lg") or nlp = English()

In [ ]:
nlp = spacy.load("en_core_web_lg")
#nlp = English()

In [ ]:
names_dict = {'name': [],
              'original_sentence': [],
              'named_sentence':[],
              'final_sentence':[]}

for index, row in test_df.iterrows():
  for name in names_list:


    original_sentence = row['Sentence']
    named_sentence = original_sentence.replace("***", name)

    final_sentence = remove_name(named_sentence, check_name = True, check_org = False, check_gpe = False)

    names_dict['name'].append(name)
    names_dict['original_sentence'].append(original_sentence)
    names_dict['named_sentence'].append(named_sentence)
    names_dict['final_sentence'].append(final_sentence)


result_df = pd.DataFrame(names_dict)

In [ ]:
result_df.final_sentence.to_list()

["I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Ariel.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Lorelai.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Elijah.",
 "I didn't like being in the same group as Zakary.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Stanley.",
 "I didn't like being in the same 

In [ ]:
result_df.to_csv("name_post1960_full_rn_results_df_20211023.csv")

### Results analysis

In [ ]:
%%R
library(tidyverse)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
%%R 
results_df <- read_csv("name_post1960_full_rn_results_df_20211023.csv")

R[write to console]: New names:
* `` -> ...1



Rows: 348960 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): name, original_sentence, named_sentence, final_sentence
dbl (1): ...1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
%%R 
results_df

# A tibble: 348,960 × 5
    ...1 name    original_sentence     named_sentence       final_sentence      
   <dbl> <chr>   <chr>                 <chr>                <chr>               
 1     0 Brock   I didn't like being … I didn't like being… I didn't like being…
 2     1 Paulo   I didn't like being … I didn't like being… I didn't like being…
 3     2 Brennen I didn't like being … I didn't like being… I didn't like being…
 4     3 Damian  I didn't like being … I didn't like being… I didn't like being…
 5     4 Ariel   I didn't like being … I didn't like being… I didn't like being…
 6     5 Rogers  I didn't like being … I didn't like being… I didn't like being…
 7     6 Astrid  I didn't like being … I didn't like being… I didn't like being…
 8     7 Lorelai I didn't like being … I didn't like being… I didn't like being…
 9     8 Abdul   I didn't like being … I didn't like being… I didn't like being…
10     9 Cathy   I didn't like being … I didn't like being… I didn't like being…
# … 

In [ ]:
%%R

results_df <- results_df %>%
mutate(name_removed_flag = case_when(str_detect(final_sentence, "\\*\\*\\*") == 1 ~ 1, 
                                     str_detect(final_sentence, "\\*\\*\\*") == 0 ~ 0))

In [ ]:
%%R 
results_df

# A tibble: 348,960 × 6
    ...1 name    original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>   <chr>            <chr>          <chr>                     <dbl>
 1     0 Brock   I didn't like b… I didn't like… I didn't like…                1
 2     1 Paulo   I didn't like b… I didn't like… I didn't like…                1
 3     2 Brennen I didn't like b… I didn't like… I didn't like…                1
 4     3 Damian  I didn't like b… I didn't like… I didn't like…                1
 5     4 Ariel   I didn't like b… I didn't like… I didn't like…                0
 6     5 Rogers  I didn't like b… I didn't like… I didn't like…                1
 7     6 Astrid  I didn't like b… I didn't like… I didn't like…                1
 8     7 Lorelai I didn't like b… I didn't like… I didn't like…                0
 9     8 Abdul   I didn't like b… I didn't like… I didn't like…                1
10     9 Cathy   I didn't like b… I didn't like… I didn't like…                1
# … 

In [ ]:
%%R

mean(results_df$name_removed_flag)

[1] 0.766681


In [ ]:
%%R

failed_names_df <- results_df %>%
filter(name_removed_flag == 0)

In [ ]:
%%R 
failed_names_df

# A tibble: 41,773 × 6
    ...1 name      original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>     <chr>            <chr>          <chr>                     <dbl>
 1     5 Isaiah    I didn't like b… I didn't like… I didn't like…                0
 2     8 Rianna    I didn't like b… I didn't like… I didn't like…                0
 3    10 Katrina   I didn't like b… I didn't like… I didn't like…                0
 4    26 Karlie    I didn't like b… I didn't like… I didn't like…                0
 5    39 Jackeline I didn't like b… I didn't like… I didn't like…                0
 6    69 Cristian  I didn't like b… I didn't like… I didn't like…                0
 7    89 Uriel     I didn't like b… I didn't like… I didn't like…                0
 8    92 Elouise   I didn't like b… I didn't like… I didn't like…                0
 9   124 Karma     I didn't like b… I didn't like… I didn't like…                0
10   158 Jesus     I didn't like b… I didn't like… I didn't like

In [ ]:
%%R 
failed_names_df %>%
distinct(name)

# A tibble: 2,808 × 1
   name     
   <chr>    
 1 Isaiah   
 2 Rianna   
 3 Katrina  
 4 Karlie   
 5 Jackeline
 6 Cristian 
 7 Uriel    
 8 Elouise  
 9 Karma    
10 Jesus    
# … with 2,798 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name)


# A tibble: 2,808 × 2
   name         n
   <chr>    <int>
 1 Aaden        7
 2 Aaliyah      1
 3 Aarav       67
 4 Abbey        1
 5 Abbigail     9
 6 Abdiel      17
 7 Abdullah     1
 8 Abel        49
 9 Abigale     24
10 Abigayle    12
# … with 2,798 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name) %>%
arrange(desc(n))

# A tibble: 2,808 × 2
   name         n
   <chr>    <int>
 1 April       78
 2 August      78
 3 German      78
 4 Heaven      78
 5 January     78
 6 Jesus       78
 7 June        78
 8 Karma       78
 9 Precious    78
10 Summer      78
# … with 2,798 more rows


## Test top 500 names since 1950

Create the dataframe with only top 500 names per year and not top 1000 names per year

In [ ]:
%%R 
names_df <- read_csv("baby-names_hadley_20211022.csv")

Rows: 258000 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): name, sex
dbl (2): year, percent

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
%%R 
names(names_df)

[1] "year"    "name"    "percent" "sex"    


In [ ]:
%%R 
names_ranking_df <- names_df %>% 
group_by(year, sex) %>% 
  mutate(name_yr_sex_rank = order(percent, decreasing = TRUE))

names_ranking_df

# A tibble: 258,000 × 5
# Groups:   year, sex [258]
    year name    percent sex   name_yr_sex_rank
   <dbl> <chr>     <dbl> <chr>            <int>
 1  1880 John     0.0815 boy                  1
 2  1880 William  0.0805 boy                  2
 3  1880 James    0.0501 boy                  3
 4  1880 Charles  0.0452 boy                  4
 5  1880 George   0.0433 boy                  5
 6  1880 Frank    0.0274 boy                  6
 7  1880 Joseph   0.0222 boy                  7
 8  1880 Thomas   0.0214 boy                  8
 9  1880 Henry    0.0206 boy                  9
10  1880 Robert   0.0204 boy                 10
# … with 257,990 more rows


In [ ]:
%%R 
names_ranking_df %>% write_csv("baby_names_ranking_df.csv")

In [ ]:
names_df = pd.read_csv("baby_names_ranking_df.csv")

In [ ]:
names_df.columns

Index(['year', 'name', 'percent', 'sex', 'name_yr_sex_rank'], dtype='object')

In [ ]:
names_df

,year,name,percent,sex,name_yr_sex_rank
0,1880,John,0.081541,boy,1
1,1880,William,0.080511,boy,2
2,1880,James,0.050057,boy,3
3,1880,Charles,0.045167,boy,4
4,1880,George,0.043292,boy,5
...,...,...,...,...,...
257995,2008,Carleigh,0.000128,girl,996
257996,2008,Iyana,0.000128,girl,997
257997,2008,Kenley,0.000127,girl,998
257998,2008,Sloane,0.000127,girl,999


In [ ]:
names_df[names_df['year'] > 1949].head(30)

,year,name,percent,sex,name_yr_sex_rank
70000,1950,James,0.047336,boy,1
70001,1950,Robert,0.045930,boy,2
70002,1950,John,0.043655,boy,3
70003,1950,Michael,0.035810,boy,4
70004,1950,David,0.033382,boy,5
70005,1950,William,0.033372,boy,6
70006,1950,Richard,0.028027,boy,7
70007,1950,Thomas,0.025070,boy,8
70008,1950,Charles,0.021495,boy,9
70009,1950,Gary,0.018547,boy,10


In [ ]:
names_500_post1950_df = names_df[(names_df['year'] > 1949) & (names_df['name_yr_sex_rank'] < 501)]
names_500_post1950_df

,year,name,percent,sex,name_yr_sex_rank
70000,1950,James,0.047336,boy,1
70001,1950,Robert,0.045930,boy,2
70002,1950,John,0.043655,boy,3
70003,1950,Michael,0.035810,boy,4
70004,1950,David,0.033382,boy,5
...,...,...,...,...,...
257495,2008,Cheyanne,0.000304,girl,496
257496,2008,Haylie,0.000304,girl,497
257497,2008,Patricia,0.000303,girl,498
257498,2008,Anne,0.000301,girl,499


In [ ]:
names_list = names_500_post1950_df.name.to_list()

In [ ]:
len(names_list)

59000

Select only the unique names from the names list

In [ ]:
names_set = set(names_list)

In [ ]:
names_list = list(names_set)

In [ ]:
len(names_list) # only 2285 unique names

2285

In [ ]:
names_list

['Phillip',
 'David',
 'Becky',
 'Benita',
 'Nelda',
 'Catina',
 'Marquita',
 'Patrick',
 'Katherine',
 'Jerrold',
 'Johnathan',
 'Lavonne',
 'Antonia',
 'Darnell',
 'Keshia',
 'Marva',
 'Grover',
 'Tiana',
 'Dayana',
 'Shelia',
 'Moses',
 'Diamond',
 'Roger',
 'Gayle',
 'Melodie',
 'Charles',
 'Jack',
 'Annika',
 'Danica',
 'Althea',
 'Ben',
 'Lena',
 'Mollie',
 'Ciera',
 'Aubrey',
 'Jordon',
 'Milton',
 'Traci',
 'Roberta',
 'Jackson',
 'Terrance',
 'Gage',
 'Piper',
 'Asher',
 'Mayra',
 'Jamarion',
 'Iris',
 'Edwin',
 'Laney',
 'Rose',
 'Lavar',
 'Arthur',
 'Barbara',
 'Ward',
 'Dudley',
 'George',
 'Arturo',
 'Laurie',
 'Carly',
 'Kirk',
 'Joe',
 'Erick',
 'Leila',
 'Vonda',
 'Keith',
 'Edith',
 'Lucy',
 'Brynn',
 'Ahmad',
 'Tiffanie',
 'Raven',
 'Brett',
 'Patsy',
 'Devante',
 'Jana',
 'Raegan',
 'Wanda',
 'Victoria',
 'Harmony',
 'Tammie',
 'Alessandra',
 'Mack',
 'Lynda',
 'Cheyanne',
 'Eula',
 'Debora',
 'Alan',
 'Ursula',
 'Tawana',
 'Cyrus',
 'Blake',
 'Dominick',
 'Alberto',

In [ ]:
# short sentence list to make sure test cycle working

test_sent_df = sentence_df.head(10)
test_sent_df

,Sentence,sentence type
1,I didn't like being in the same group as ***.,name_only
2,Dr. *** has a weird way of talking to the rest...,name_only
3,I thought *** was a great teacher,name_only
4,*** works hard in class and always shows up on...,name_only
5,I really wish that *** wasn't on my team,name_only
6,The note's from ***'s class were great,name_only
7,I was told by *** that we didn't need to finis...,name_only
8,I went to office hours ever week and got help ...,name_only
9,The textbook by *** was tough to read,name_only
10,He told *** that she was going to need a new p...,name_only


In [ ]:
# short test to make sure test cycle works

#test_names_list = names_list[-10:]
#test_names_list

### Test section

We will use the following code to test the name removal process

#### Creating the Names Only dataset for post 1950 names and nlp = spacy.load("en_core_web_lg")



In [ ]:
# Test the remove names function with either nlp = spacy.load("en_core_web_lg") or nlp = English()

In [ ]:
nlp = spacy.load("en_core_web_lg")
#nlp = English()

In [ ]:
names_dict = {'name': [],
              'original_sentence': [],
              'named_sentence':[],
              'final_sentence':[]}

for index, row in test_df.iterrows():
  for name in names_list:


    original_sentence = row['Sentence']
    named_sentence = original_sentence.replace("***", name)

    final_sentence = remove_name(named_sentence, check_name = True, check_org = False, check_gpe = False)

    names_dict['name'].append(name)
    names_dict['original_sentence'].append(original_sentence)
    names_dict['named_sentence'].append(named_sentence)
    names_dict['final_sentence'].append(final_sentence)


result_df = pd.DataFrame(names_dict)

In [ ]:
result_df.final_sentence.to_list()

["I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Nelda.",
 "I didn't like being in the same group as Catina.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as Patrick.",
 "I didn't like being in the same group as Katherine.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same group as ***.",
 "I didn't like being in the same g

In [ ]:
result_df.to_csv("name_post1950_500_rn_results_df_20211023.csv")

### Results analysis - Top 500 names since 1950

In [ ]:
%%R
library(tidyverse)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
%%R 
results_df <- read_csv("name_post1950_500_rn_results_df_20211023.csv")

R[write to console]: New names:
* `` -> ...1



Rows: 182800 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): name, original_sentence, named_sentence, final_sentence
dbl (1): ...1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
%%R 
results_df

# A tibble: 182,800 × 5
    ...1 name      original_sentence     named_sentence      final_sentence     
   <dbl> <chr>     <chr>                 <chr>               <chr>              
 1     0 Phillip   I didn't like being … I didn't like bein… I didn't like bein…
 2     1 David     I didn't like being … I didn't like bein… I didn't like bein…
 3     2 Becky     I didn't like being … I didn't like bein… I didn't like bein…
 4     3 Benita    I didn't like being … I didn't like bein… I didn't like bein…
 5     4 Nelda     I didn't like being … I didn't like bein… I didn't like bein…
 6     5 Catina    I didn't like being … I didn't like bein… I didn't like bein…
 7     6 Marquita  I didn't like being … I didn't like bein… I didn't like bein…
 8     7 Patrick   I didn't like being … I didn't like bein… I didn't like bein…
 9     8 Katherine I didn't like being … I didn't like bein… I didn't like bein…
10     9 Jerrold   I didn't like being … I didn't like bein… I didn't like bein…
# … 

In [ ]:
%%R

results_df <- results_df %>%
mutate(name_removed_flag = case_when(str_detect(final_sentence, "\\*\\*\\*|\\[\\[\\[|\\-\\-\\-") == 1 ~ 1, 
                                     str_detect(final_sentence, "\\*\\*\\*|\\[\\[\\[") == 0 ~ 0))

In [ ]:
%%R 
results_df

# A tibble: 182,800 × 6
    ...1 name      original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>     <chr>            <chr>          <chr>                     <dbl>
 1     0 Phillip   I didn't like b… I didn't like… I didn't like…                1
 2     1 David     I didn't like b… I didn't like… I didn't like…                1
 3     2 Becky     I didn't like b… I didn't like… I didn't like…                1
 4     3 Benita    I didn't like b… I didn't like… I didn't like…                1
 5     4 Nelda     I didn't like b… I didn't like… I didn't like…                0
 6     5 Catina    I didn't like b… I didn't like… I didn't like…                0
 7     6 Marquita  I didn't like b… I didn't like… I didn't like…                1
 8     7 Patrick   I didn't like b… I didn't like… I didn't like…                0
 9     8 Katherine I didn't like b… I didn't like… I didn't like…                0
10     9 Jerrold   I didn't like b… I didn't like… I didn't lik

In [ ]:
%%R 
results_df %>% 
filter(name_removed_flag == 0) %>% 
select(final_sentence)

# A tibble: 88,859 × 1
   final_sentence                                      
   <chr>                                               
 1 I didn't like being in the same group as Sanjuanita.
 2 I didn't like being in the same group as Ariel.     
 3 I didn't like being in the same group as Lorelai.   
 4 I didn't like being in the same group as Elijah.    
 5 I didn't like being in the same group as Zakary.    
 6 I didn't like being in the same group as Stanley.   
 7 I didn't like being in the same group as Lara.      
 8 I didn't like being in the same group as India.     
 9 I didn't like being in the same group as Douglass.  
10 I didn't like being in the same group as Franklin.  
# … with 88,849 more rows


In [ ]:
%%R

mean(results_df$name_removed_flag)

[1] 0.8292123


In [ ]:
%%R

failed_names_df <- results_df %>%
filter(name_removed_flag == 0)

In [ ]:
%%R 
failed_names_df

# A tibble: 41,773 × 6
    ...1 name      original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>     <chr>            <chr>          <chr>                     <dbl>
 1     5 Isaiah    I didn't like b… I didn't like… I didn't like…                0
 2     8 Rianna    I didn't like b… I didn't like… I didn't like…                0
 3    10 Katrina   I didn't like b… I didn't like… I didn't like…                0
 4    26 Karlie    I didn't like b… I didn't like… I didn't like…                0
 5    39 Jackeline I didn't like b… I didn't like… I didn't like…                0
 6    69 Cristian  I didn't like b… I didn't like… I didn't like…                0
 7    89 Uriel     I didn't like b… I didn't like… I didn't like…                0
 8    92 Elouise   I didn't like b… I didn't like… I didn't like…                0
 9   124 Karma     I didn't like b… I didn't like… I didn't like…                0
10   158 Jesus     I didn't like b… I didn't like… I didn't like

In [ ]:
%%R 
failed_names_df %>%
distinct(name)

# A tibble: 2,808 × 1
   name     
   <chr>    
 1 Isaiah   
 2 Rianna   
 3 Katrina  
 4 Karlie   
 5 Jackeline
 6 Cristian 
 7 Uriel    
 8 Elouise  
 9 Karma    
10 Jesus    
# … with 2,798 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name)


# A tibble: 2,808 × 2
   name         n
   <chr>    <int>
 1 Aaden        7
 2 Aaliyah      1
 3 Aarav       67
 4 Abbey        1
 5 Abbigail     9
 6 Abdiel      17
 7 Abdullah     1
 8 Abel        49
 9 Abigale     24
10 Abigayle    12
# … with 2,798 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name) %>%
arrange(desc(n))

# A tibble: 2,808 × 2
   name         n
   <chr>    <int>
 1 April       78
 2 August      78
 3 German      78
 4 Heaven      78
 5 January     78
 6 Jesus       78
 7 June        78
 8 Karma       78
 9 Precious    78
10 Summer      78
# … with 2,798 more rows


## Test 5000 full names by sampling top 500 names from 1960 per year per gender dataset and surnames dataset

In [ ]:
first_names_df = pd.read_csv("baby_names_ranking_df.csv")

In [ ]:
first_names_df.columns

Index(['year', 'name', 'percent', 'sex', 'name_yr_sex_rank'], dtype='object')

In [ ]:
first_names_df

,year,name,percent,sex,name_yr_sex_rank
0,1880,John,0.081541,boy,1
1,1880,William,0.080511,boy,2
2,1880,James,0.050057,boy,3
3,1880,Charles,0.045167,boy,4
4,1880,George,0.043292,boy,5
...,...,...,...,...,...
257995,2008,Carleigh,0.000128,girl,996
257996,2008,Iyana,0.000128,girl,997
257997,2008,Kenley,0.000127,girl,998
257998,2008,Sloane,0.000127,girl,999


In [ ]:
names_df[names_df['year'] > 1959].head(15)

,year,name,percent,sex
80000,1960,David,0.039669,boy
80001,1960,Michael,0.038868,boy
80002,1960,James,0.035483,boy
80003,1960,John,0.035145,boy
80004,1960,Robert,0.033415,boy
80005,1960,Mark,0.027121,boy
80006,1960,William,0.022783,boy
80007,1960,Richard,0.020106,boy
80008,1960,Thomas,0.018145,boy
80009,1960,Steven,0.015651,boy


In [ ]:
first_names_500_post1960_df = first_names_df[(first_names_df['year'] > 1959) & (first_names_df['name_yr_sex_rank'] < 501)]
first_names_500_post1960_df

,year,name,percent,sex,name_yr_sex_rank
80000,1960,David,0.039669,boy,1
80001,1960,Michael,0.038868,boy,2
80002,1960,James,0.035483,boy,3
80003,1960,John,0.035145,boy,4
80004,1960,Robert,0.033415,boy,5
...,...,...,...,...,...
257495,2008,Cheyanne,0.000304,girl,496
257496,2008,Haylie,0.000304,girl,497
257497,2008,Patricia,0.000303,girl,498
257498,2008,Anne,0.000301,girl,499


In [ ]:
first_names_list = first_names_500_post1960_df.name.to_list()

In [ ]:
len(first_names_list)

49000

In [ ]:
first_names_list = list(set(first_names_list))

In [ ]:
len(first_names_list)

2144

Create last names list

In [ ]:
surnames_list = surnames_df.name.to_list()

In [ ]:
len(surnames_list)

1000

In [ ]:
surnames_list[:8]

['SMITH', 'JOHNSON', 'WILLIAMS', 'BROWN', 'JONES', 'MILLER', 'DAVIS', 'GARCIA']

Need to change the case of last names from all upper case

In [ ]:
surnames_list = [name.lower().capitalize() for name in surnames_list]

In [ ]:
surnames_list[:8]

['Smith', 'Johnson', 'Williams', 'Brown', 'Jones', 'Miller', 'Davis', 'Garcia']

Create names to pass into sentence

In [ ]:
full_name_list = []
for first_name in first_names_list:
  for surname in surnames_list:
    full_name = first_name + " " + surname
    full_name_list.append(full_name)

In [ ]:
full_name_list[:10]

['Juliet Smith',
 'Juliet Johnson',
 'Juliet Williams',
 'Juliet Brown',
 'Juliet Jones',
 'Juliet Miller',
 'Juliet Davis',
 'Juliet Garcia',
 'Juliet Rodriguez',
 'Juliet Wilson']

Create list of 5000 unique full names to test

In [ ]:
import random

In [ ]:
random.seed(123)
k = 5000
test_full_name_list = sample(full_name_list, k=k)

In [ ]:
test_full_name_list[:10]

['Rosalie Delacruz',
 'Christen Stein',
 'Billie Mcclain',
 'Sammie Gutierrez',
 'Keshia Garcia',
 'Zackery Odonnell',
 'Margaret Perez',
 'Kiersten Meyer',
 'Lyndsay Lyons',
 'Annie Christensen']

#### Creating the Names Only dataset for post 1950 names and nlp = spacy.load("en_core_web_lg")



In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
names_dict = {'name': [],
              'original_sentence': [],
              'named_sentence':[],
              'final_sentence':[]}

for index, row in test_df.iterrows():
  for name in test_full_name_list:


    original_sentence = row['Sentence']
    named_sentence = original_sentence.replace("***", name)

    final_sentence = remove_name(named_sentence, check_name = True, check_org = False, check_gpe = False)

    names_dict['name'].append(name)
    names_dict['original_sentence'].append(original_sentence)
    names_dict['named_sentence'].append(named_sentence)
    names_dict['final_sentence'].append(final_sentence)


result_df = pd.DataFrame(names_dict)

In [ ]:
result_df.to_csv("full_name_post1960_5000_rn_results_df_20211024.csv")

In [ ]:
result_df.final_sentence.to_list()

### Results analysis - 5000 full names

In [ ]:
%%R
library(tidyverse)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
%%R 
results_df <- read_csv("full_name_post1960_5000_rn_results_df_20211024.csv")

R[write to console]: New names:
* `` -> ...1



Rows: 400000 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): name, original_sentence, named_sentence, final_sentence
dbl (1): ...1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
%%R 
results_df

# A tibble: 400,000 × 5
    ...1 name              original_sentence  named_sentence    final_sentence  
   <dbl> <chr>             <chr>              <chr>             <chr>           
 1     0 Rosalie Delacruz  I didn't like bei… I didn't like be… I didn't like b…
 2     1 Christen Stein    I didn't like bei… I didn't like be… I didn't like b…
 3     2 Billie Mcclain    I didn't like bei… I didn't like be… I didn't like b…
 4     3 Sammie Gutierrez  I didn't like bei… I didn't like be… I didn't like b…
 5     4 Keshia Garcia     I didn't like bei… I didn't like be… I didn't like b…
 6     5 Zackery Odonnell  I didn't like bei… I didn't like be… I didn't like b…
 7     6 Margaret Perez    I didn't like bei… I didn't like be… I didn't like b…
 8     7 Kiersten Meyer    I didn't like bei… I didn't like be… I didn't like b…
 9     8 Lyndsay Lyons     I didn't like bei… I didn't like be… I didn't like b…
10     9 Annie Christensen I didn't like bei… I didn't like be… I didn't like b…
# … 

In [ ]:
%%R

results_df <- results_df %>%
mutate(name_removed_flag = case_when(str_detect(final_sentence, "\\*\\*\\*|\\[\\[\\[|\\-\\-\\-") == 1 ~ 1, 
                                     str_detect(final_sentence, "\\*\\*\\*|\\[\\[\\[") == 0 ~ 0))

In [ ]:
%%R 
results_df

# A tibble: 400,000 × 6
    ...1 name              original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>             <chr>            <chr>          <chr>                     <dbl>
 1     0 Rosalie Delacruz  I didn't like b… I didn't like… I didn't like…                1
 2     1 Christen Stein    I didn't like b… I didn't like… I didn't like…                1
 3     2 Billie Mcclain    I didn't like b… I didn't like… I didn't like…                1
 4     3 Sammie Gutierrez  I didn't like b… I didn't like… I didn't like…                1
 5     4 Keshia Garcia     I didn't like b… I didn't like… I didn't like…                1
 6     5 Zackery Odonnell  I didn't like b… I didn't like… I didn't like…                1
 7     6 Margaret Perez    I didn't like b… I didn't like… I didn't like…                1
 8     7 Kiersten Meyer    I didn't like b… I didn't like… I didn't like…                1
 9     8 Lyndsay Lyons     I didn't like b… I didn't like… I didn'

In [ ]:
%%R 
results_df %>% 
filter(name_removed_flag == 0) %>% 
select(final_sentence)

# A tibble: 14,526 × 1
   final_sentence                                             
   <chr>                                                      
 1 I didn't like being in the same group as Marina English.   
 2 I didn't like being in the same group as Kirstie Avery.    
 3 I didn't like being in the same group as Esmeralda Mcbride.
 4 I didn't like being in the same group as Trace Rivas.      
 5 I didn't like being in the same group as America Wall.     
 6 I didn't like being in the same group as Marquise Franklin.
 7 I didn't like being in the same group as Reba Cross.       
 8 I didn't like being in the same group as Daquan Waters.    
 9 I didn't like being in the same group as Homer Mcbride.    
10 I didn't like being in the same group as Regina Sellers.   
# … with 14,516 more rows


In [ ]:
%%R

mean(results_df$name_removed_flag)

[1] 0.963685


In [ ]:
%%R

failed_names_df <- results_df %>%
filter(name_removed_flag == 0)

In [ ]:
%%R 
failed_names_df

# A tibble: 14,526 × 6
    ...1 name              original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>             <chr>            <chr>          <chr>                     <dbl>
 1    10 Marina English    I didn't like b… I didn't like… I didn't like…                0
 2    34 Kirstie Avery     I didn't like b… I didn't like… I didn't like…                0
 3    46 Esmeralda Mcbride I didn't like b… I didn't like… I didn't like…                0
 4    49 Trace Rivas       I didn't like b… I didn't like… I didn't like…                0
 5    89 America Wall      I didn't like b… I didn't like… I didn't like…                0
 6    96 Marquise Franklin I didn't like b… I didn't like… I didn't like…                0
 7   109 Reba Cross        I didn't like b… I didn't like… I didn't like…                0
 8   136 Daquan Waters     I didn't like b… I didn't like… I didn't like…                0
 9   209 Homer Mcbride     I didn't like b… I didn't like… I didn't

In [ ]:
%%R 
failed_names_df %>%
distinct(name)

# A tibble: 1,208 × 1
   name             
   <chr>            
 1 Marina English   
 2 Kirstie Avery    
 3 Esmeralda Mcbride
 4 Trace Rivas      
 5 America Wall     
 6 Marquise Franklin
 7 Reba Cross       
 8 Daquan Waters    
 9 Homer Mcbride    
10 Regina Sellers   
# … with 1,198 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name)


# A tibble: 1,208 × 2
   name                 n
   <chr>            <int>
 1 Aaden Cantu          1
 2 Aaden Farmer         1
 3 Aaliyah Medina       9
 4 Abbigail Douglas     7
 5 Abril Howell        29
 6 Abril Hughes        29
 7 Abril Sanford        2
 8 Addison Carney      11
 9 Addison Erickson    13
10 Addison Gonzalez     3
# … with 1,198 more rows


In [ ]:
%%R 
failed_names_df %>%
count(name) %>%
arrange(desc(n))

# A tibble: 1,208 × 2
   name               n
   <chr>          <int>
 1 America Wall      77
 2 Asia Sharp        77
 3 Autumn Rivers     77
 4 Cristal Rich      77
 5 Precious Mayo     77
 6 Daphne Weeks      76
 7 Ebony Valencia    76
 8 Lexus House       76
 9 Salvador Weeks    76
10 Athena French     75
# … with 1,198 more rows


## Negative cases test (with no names)

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
import random

In [ ]:
# set seed for reproducibility
random.seed(123)
# check to make sure sampling working if wanting to sample rather than do exhaustive search
sample(["he", "she", "we", "you", "you all", "they"], 2)

['he', 'we']

In [ ]:
# for negative cases, just replace name placeholder with a pronoun

pronouns = ["he", "she", "we", "you", "you all", "they", "I", "me", "him", "her", "them", "us"]

names_dict = {'name': [],
              'original_sentence': [],
              'named_sentence':[],
              'final_sentence':[]}

for index, row in test_df.iterrows():
  for pronoun in pronouns:


    original_sentence = row['Sentence']
    named_sentence = original_sentence.replace("***", pronoun)

    final_sentence = remove_name(named_sentence, check_name = True, check_org = False, check_gpe = False)

    names_dict['name'].append(pronoun)
    names_dict['original_sentence'].append(original_sentence)
    names_dict['named_sentence'].append(named_sentence)
    names_dict['final_sentence'].append(final_sentence)


result_df = pd.DataFrame(names_dict)

In [ ]:
result_df.to_csv("no_name_results_df_20211026.csv")

In [ ]:
result_df.final_sentence.to_list()

["I didn't like being in the same group as he.",
 "I didn't like being in the same group as she.",
 "I didn't like being in the same group as we.",
 "I didn't like being in the same group as you.",
 "I didn't like being in the same group as you all.",
 "I didn't like being in the same group as they.",
 "I didn't like being in the same group as I.",
 "I didn't like being in the same group as me.",
 "I didn't like being in the same group as him.",
 "I didn't like being in the same group as her.",
 "I didn't like being in the same group as them.",
 "I didn't like being in the same group as us.",
 'Dr. he has a weird way of talking to the rest of the class',
 'Dr. she has a weird way of talking to the rest of the class',
 'Dr. we has a weird way of talking to the rest of the class',
 'Dr. you has a weird way of talking to the rest of the class',
 'Dr. you all has a weird way of talking to the rest of the class',
 'Dr. they has a weird way of talking to the rest of the class',
 'Dr. I has a

### Results analysis - no names

In [ ]:
%%R
library(tidyverse)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [ ]:
%%R 
results_df <- read_csv("no_name_results_df_20211026.csv")

R[write to console]: New names:
* `` -> ...1



Rows: 960 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): name, original_sentence, named_sentence, final_sentence
dbl (1): ...1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
%%R 
results_df

# A tibble: 960 × 5
    ...1 name    original_sentence     named_sentence       final_sentence      
   <dbl> <chr>   <chr>                 <chr>                <chr>               
 1     0 he      I didn't like being … I didn't like being… I didn't like being…
 2     1 she     I didn't like being … I didn't like being… I didn't like being…
 3     2 we      I didn't like being … I didn't like being… I didn't like being…
 4     3 you     I didn't like being … I didn't like being… I didn't like being…
 5     4 you all I didn't like being … I didn't like being… I didn't like being…
 6     5 they    I didn't like being … I didn't like being… I didn't like being…
 7     6 I       I didn't like being … I didn't like being… I didn't like being…
 8     7 me      I didn't like being … I didn't like being… I didn't like being…
 9     8 him     I didn't like being … I didn't like being… I didn't like being…
10     9 her     I didn't like being … I didn't like being… I didn't like being…
# … with

In [ ]:
%%R

results_df <- results_df %>%
mutate(name_removed_flag = case_when(str_detect(final_sentence, "\\*\\*\\*") == 1 ~ 1, 
                                     str_detect(final_sentence, "\\*\\*\\*") == 0 ~ 0))

In [ ]:
%%R 
results_df

# A tibble: 960 × 6
    ...1 name    original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>   <chr>            <chr>          <chr>                     <dbl>
 1     0 he      I didn't like b… I didn't like… I didn't like…                0
 2     1 she     I didn't like b… I didn't like… I didn't like…                0
 3     2 we      I didn't like b… I didn't like… I didn't like…                0
 4     3 you     I didn't like b… I didn't like… I didn't like…                0
 5     4 you all I didn't like b… I didn't like… I didn't like…                0
 6     5 they    I didn't like b… I didn't like… I didn't like…                0
 7     6 I       I didn't like b… I didn't like… I didn't like…                0
 8     7 me      I didn't like b… I didn't like… I didn't like…                0
 9     8 him     I didn't like b… I didn't like… I didn't like…                0
10     9 her     I didn't like b… I didn't like… I didn't like…                0
# … with

In [ ]:
%%R 
results_df %>% 
filter(name_removed_flag == 0) %>% 
select(final_sentence)

# A tibble: 946 × 1
   final_sentence                                   
   <chr>                                            
 1 I didn't like being in the same group as he.     
 2 I didn't like being in the same group as she.    
 3 I didn't like being in the same group as we.     
 4 I didn't like being in the same group as you.    
 5 I didn't like being in the same group as you all.
 6 I didn't like being in the same group as they.   
 7 I didn't like being in the same group as I.      
 8 I didn't like being in the same group as me.     
 9 I didn't like being in the same group as him.    
10 I didn't like being in the same group as her.    
# … with 936 more rows


In [ ]:
%%R

mean(results_df$name_removed_flag)

[1] 0.01458333


This result means the test was 1 - 0.014 = 98.6 % specific (identifying true negatives)

In [ ]:
%%R
# flip to look at the ones flagged with 1
failed_names_df <- results_df %>%
filter(name_removed_flag == 1)

In [ ]:
%%R 
failed_names_df

# A tibble: 14 × 6
    ...1 name    original_senten… named_sentence final_sentence name_removed_fl…
   <dbl> <chr>   <chr>            <chr>          <chr>                     <dbl>
 1   402 I       I was told not … I was told no… I was told no…                1
 2   498 I       I distinctly re… I distinctly … I distinctly …                1
 3   852 he      *** is not rela… he is not rel… he is not rel…                1
 4   853 she     *** is not rela… she is not re… she is not re…                1
 5   854 we      *** is not rela… we is not rel… we is not rel…                1
 6   855 you     *** is not rela… you is not re… you is not re…                1
 7   856 you all *** is not rela… you all is no… you all is no…                1
 8   857 they    *** is not rela… they is not r… they is not r…                1
 9   858 I       *** is not rela… I is not rela… I is not rela…                1
10   859 me      *** is not rela… me is not rel… me is not rel…                1
11   860 

In [ ]:
%%R 
failed_names_df$final_sentence

 [1] "I was told not to interrupt ***"                    
 [2] "I distinctly remember the first time I ever met ***"
 [3] "he is not related to that person but knows ***"     
 [4] "she is not related to that person but knows ***"    
 [5] "we is not related to that person but knows ***"     
 [6] "you is not related to that person but knows ***"    
 [7] "you all is not related to that person but knows ***"
 [8] "they is not related to that person but knows ***"   
 [9] "I is not related to that person but knows ***"      
[10] "me is not related to that person but knows ***"     
[11] "him is not related to that person but knows ***"    
[12] "her is not related to that person but knows ***"    
[13] "them is not related to that person but knows ***"   
[14] "us is not related to that person but knows ***"     


Apparently "Karate" was recognized as a name

In [ ]:
%%R 
failed_names_df %>%
distinct(name)

# A tibble: 12 × 1
   name   
   <chr>  
 1 I      
 2 he     
 3 she    
 4 we     
 5 you    
 6 you all
 7 they   
 8 me     
 9 him    
10 her    
11 them   
12 us     


In [ ]:
%%R 
failed_names_df %>%
count(name)


# A tibble: 12 × 2
   name        n
   <chr>   <int>
 1 he          2
 2 her         2
 3 him         2
 4 I           4
 5 me          2
 6 she         2
 7 them        2
 8 they        2
 9 us          2
10 we          2
11 you         2
12 you all     2


In [ ]:
%%R 
failed_names_df %>%
count(name) %>%
arrange(desc(n))

# A tibble: 12 × 2
   name        n
   <chr>   <int>
 1 I           4
 2 he          2
 3 her         2
 4 him         2
 5 me          2
 6 she         2
 7 them        2
 8 they        2
 9 us          2
10 we          2
11 you         2
12 you all     2
